In [186]:
import torch
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [187]:
# Read .csv
data = pd.read_csv('data.csv')
data.columns

Index([&#39;malignant&#39;, &#39;bilateral&#39;, &#39;age&#39;, &#39;month&#39;, &#39;bmi&#39;, &#39;diabetes&#39;,
       &#39;hypertension&#39;, &#39;surgery&#39;, &#39;smoking&#39;, &#39;family&#39;, &#39;menarche&#39;, &#39;menopause&#39;,
       &#39;miscarriage&#39;, &#39;pregnancy&#39;, &#39;childbearing&#39;, &#39;nd_blood&#39;, &#39;nd_milk&#39;,
       &#39;nd_serous&#39;, &#39;nd_change&#39;, &#39;nd_number&#39;, &#39;nd_mass&#39;, &#39;nd_position&#39;,
       &#39;nd_distance&#39;, &#39;nd_shape&#39;, &#39;nd_surface&#39;, &#39;nd_dispersion&#39;, &#39;nd_quantity&#39;,
       &#39;nd_distribution&#39;, &#39;nd_retraction&#39;, &#39;nd_appearance&#39;, &#39;us_ductectasia&#39;,
       &#39;us_intramass&#39;, &#39;us_cysmass&#39;, &#39;us_mass&#39;, &#39;us_size&#39;, &#39;us_position&#39;,
       &#39;us_distance&#39;, &#39;us_shape&#39;, &#39;us_boundary&#39;, &#39;us_bloodflow&#39;,
       &#39;us_calcification&#39;, &#39;us_birads&#39;, &#39;xray&#39;],
      dtype=&#39

In [188]:
# 指定标签及特征

label = data['malignant']
features = data[['bilateral', 'age', 'month', 'bmi', 'diabetes',
       'hypertension', 'surgery', 'smoking', 'family', 'menarche', 'menopause',
       'miscarriage', 'pregnancy', 'childbearing', 'nd_blood', 'nd_milk',
       'nd_serous', 'nd_change', 'nd_number', 'nd_mass', 'nd_position',
       'nd_distance', 'nd_shape', 'nd_surface', 'nd_dispersion', 'nd_quantity',
       'nd_distribution', 'nd_retraction', 'nd_appearance', 'us_ductectasia',
       'us_intramass', 'us_cysmass', 'us_mass', 'us_size', 'us_position',
       'us_distance', 'us_shape', 'us_boundary', 'us_bloodflow',
       'us_calcification', 'us_birads', 'xray']]

In [189]:
# One-Hot 编码分类数据

# features = pd.get_dummies(features)

In [190]:
# 数值标准化

# features = preprocessing.StandardScaler().fit_transform(features)

In [191]:
# 指定 x, y

X = torch.from_numpy(np.array(features)).float()
y = torch.from_numpy(np.array(label))

In [192]:
X.shape

torch.Size([355, 42])

In [193]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
seed = 42
testsize = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testsize, random_state=seed)

In [194]:
# 构建网络

net = torch.nn.Sequential(
    torch.nn.Linear(42, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 2)
)

optimizer = torch.optim.SGD(net.parameters(), lr=0.02)
loss_func = torch.nn.CrossEntropyLoss()

In [195]:
for t in range(300):
    prediction = net(X_train)
    loss = loss_func(prediction, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [196]:
y_predict = torch.max(torch.nn.functional.softmax(net(X_test)),1)[1]

In [197]:
y_predict

tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0])

In [198]:
from sklearn.metrics import confusion_matrix

In [199]:
maxtrix = confusion_matrix(y_test, y_predict)

In [200]:
maxtrix

array([[42,  4],
       [13, 12]], dtype=int64)